In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pytorch_lightning
from nlp_datasets import get_dataloader, get_dataset

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR

import gc

ModuleNotFoundError: No module named 'dataset_dir'

In [2]:
torch.cuda.empty_cache()
gc.collect()

555

In [3]:
text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    text_decoder_id,
    lagacy = False
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
for name, module in base_model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [6]:
train_dataloader = get_dataloader()
valid_dataloader = get_dataloader(split='valid')

dataset_dict:  {'prompt': '\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[보수약정금]〈북한주민인 피고들과 위임 및 보수약정을 체결한 법무법인인 원고가 위임약정에 따른 업무 수행 후 피고들을 상대로 성공보수금을 청구한 사건〉 [공2024상,727]\n판시사항:[1] 북한주민이 상속 등으로 취득한 재산 그 자체는 물론이고 그 재산을 처분한 대가로 얻은 재산을 처분하는 행위 역시 남북 주민 사이의 가족관계와 상속 등에 관한 특례법 제15조에 따라 재산관리인을 통하지 않은 경우, 무효인지 여부(적극) 및 그러한 법률행위가 재산관리인이 선임되기 전에 있었다고 하더라도 마찬가지인지 여부(적극)\n[2] 일부무효의 법리를 정한 민법 제137조에서 ‘당사자의 의사’의 의미 / 여러 개의 계약 전부가 경제적, 사실적으로 일체로서 행하여져서 하나의 계약인 것과 같은 관계에 있는 경우, 법률행위의 일부무효 법리가 적용되는지 여부(적극)\n[3] 변호사에게 계쟁 사건의 처리를 위임하는 경우, 보수 지급 및 수액에 관하여 명시적인 약정을 아니하였더라도 보수를 지급할 묵시의 약정이 있는 것으로 보아야 하는지 여부(원칙적 적극) 및 이 경우 보수액을 결정하는 방법\n[4] 갑 법무법인이 남북 주민 사이의 가족관계와 상속 등에 관한 특례법의 적용을 받는 북한주민인 을 등과 을 등의 병에 대한 친생자확인소송, 상속회복 청구소송 등에 관한 위임약정 및 ‘총상속지분의 30% 또는 이에 상응하는 금전’을 성공보수로 받는 보수약정을 체결하였고, 이후 병의 상속인들 사이에 화해가 성립하여 을 등이 상속재산을 분할받자, 갑 법무법인이 을 등을 상대로 위 보수약정에 따라 상속재산 30%에 해당하는 돈의 지급을 구하였으나, 을 등이 위 보수약정은 재산관리인을 통하지 아니하고 상속재산 등에 관하여 한 법률행위이므로, 위 특례법 제15조에 따라 무효라고 주장한 사안에서, 제반 사정에 비추어 위 보수약정

Map:   0%|          | 0/815 [00:00<?, ? examples/s]

dataset_dict:  {'prompt': '\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[손해배상(기)]〈업무집행지시자 등을 상대로 회사에 대한 손해배상을 구하는 사건〉 [공2023하,2059]\n판시사항:[1] 상법 제399조 제1항, 제414조 제1항에서 규정하고 있는 주식회사의 이사 또는 감사의 회사에 대한 임무 해태로 인한 손해배상책임에 따른 손해배상채권에 민법 제766조 제1항의 단기소멸시효가 적용되는지 여부(소극)\n[2] 상법 제401조의2 제1항에서 정한 업무집행지시자 등의 손해배상책임에 따른 손해배상채권에 일반 불법행위책임의 단기소멸시효를 규정한 민법 제766조 제1항이 적용되는지 여부(소극)\n\n    ', 'label': '\n그러므로 상고를 모두 기각하고, 상고비용은 패소자가 부담하도록 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유(상고이유서 제출기간이 지난 다음 제출된 서면 기재는 상고이유를 보충하는 범위에서)를 판단한다.\n1. 사안의 개요\n원심판결 이유와 기록에 따르면 다음 사실을 알 수 있다.\n가. ○○○호텔 주식회사는 호텔, 콘도 관리 및 운영업 등을 영위하는 회사로서 2007년경부터 ○○○호텔의 구분소유자들로부터 권한을 위탁받아 위 호텔을 운영하였다. 위 회사에 대하여 2018. 3. 16. 파산이 선고되었고(부산지방법원 2017하합4호), 원고가 위 회사의 파산관재인으로 선임되었다(이하 ○○○호텔 주식회사를 ‘파산회사’라고 한다).\n나. 피고 1은 2004. 2. 25.부터 2004. 12. 1.까지는 파산회사의 대주주로서 공동대표이사, 2007. 5. 18.부터 2008. 8. 5.까지는 파산회사의 이사로 재직하였고, 이후 2016. 5. 18.까지 파산회사의 회장으로서 실제 전반적인 운영을 담당하였다. 피고 2는 2006. 8. 2.부터 2016. 5. 18.까지 파산회사의 대표이사로 재직하였고, 피고

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

In [7]:
## ---  LoRA --- ##
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
model = get_peft_model(base_model, config)
model.print_trainable_parameters()

peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

trainable params: 4,718,592 || all params: 8,172,867,584 || trainable%: 0.0577


In [8]:
num_epochs = 20
learning_rate = 2e-5
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.85)

In [ ]:
raise ValueError()

ValueError: 

In [9]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [10]:
from tqdm import tqdm

device = "cuda"
model = model.to(device)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        with torch.autocast(device_type='cuda'):
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.detach().float()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # loss.backward()
        # optimizer.step()
        # lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(valid_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(valid_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|          | 0/815 [00:00<?, ?it/s]

100%|██████████| 91/91 [00:27<00:00,  3.32it/s]


epoch=0: train_ppl=tensor(16.1834, device='cuda:0') train_epoch_loss=tensor(2.7840, device='cuda:0') eval_ppl=tensor(10.2516, device='cuda:0') eval_epoch_loss=tensor(2.3274, device='cuda:0')


100%|██████████| 91/91 [00:27<00:00,  3.32it/s]


epoch=1: train_ppl=tensor(8.0611, device='cuda:0') train_epoch_loss=tensor(2.0870, device='cuda:0') eval_ppl=tensor(7.5395, device='cuda:0') eval_epoch_loss=tensor(2.0202, device='cuda:0')


...

epoch=7: train_ppl=tensor(2.1617, device='cuda:0') train_epoch_loss=tensor(0.7709, device='cuda:0') eval_ppl=tensor(2.4375, device='cuda:0') eval_epoch_loss=tensor(0.8910, device='cuda:0')


100%|██████████| 91/91 [00:27<00:00,  3.32it/s]


epoch=8: train_ppl=tensor(1.9085, device='cuda:0') train_epoch_loss=tensor(0.6463, device='cuda:0') eval_ppl=tensor(2.1308, device='cuda:0') eval_epoch_loss=tensor(0.7565, device='cuda:0')


100%|██████████| 91/91 [00:27<00:00,  3.32it/s]

epoch=9: train_ppl=tensor(1.7221, device='cuda:0') train_epoch_loss=tensor(0.5435, device='cuda:0') eval_ppl=tensor(1.9109, device='cuda:0') eval_epoch_loss=tensor(0.6476, device='cuda:0')


In [11]:
model.save_pretrained(peft_model_id)

/home/eahc00/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
from peft import PeftModel

# LoRA 적용 모델 불러오기
base_model = AutoModelForCausalLM.from_pretrained(text_decoder_id)
load_model = PeftModel.from_pretrained(base_model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(text_decoder_id)

ds = get_dataset(split="valid", get_prompy_only=True)


i = 2
inputs = tokenizer(ds[i]['prompt'], return_tensors="pt")
print(ds[i]['prompt'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘
        
 title:[시정명령등처분취소청구의소]〈개인정보 유출로 인한 과징금 부과처분의 취소를 구한 사건〉 [공2023하,2029]
판시사항:[1] 구 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제64조의3 제1항 각호에서 정한 행위에 대하여 부과하는 과징금의 성격 / 위 조항 제6호에서 정한 자에 대하여 과징금을 부과함으로써 박탈하고자 하는 이득 / 위 과징금 부과를 위한 관련 매출액을 산정할 때 ‘위반행위로 인하여 직접 또는 간접적으로 영향을 받는 서비스’의 범위를 판단하는 기준
[2] 구 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제64조의3 제1항에 따라 개인정보 보호조치 의무 위반에 대해 부과되는 과징금의 액수를 정할 때 고려할 사항 및 과징금의 액수가 위반행위의 내용에 비해 과중하여 사회통념상 현저하게 타당성을 잃은 경우, 과징금 부과처분이 위법한지 여부(적극)

    


In [16]:
device = "cuda"

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"])
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.


['\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[시정명령등처분취소청구의소]〈개인정보 유출로 인한 과징금 부과처분의 취소를 구한 사건〉 [공2023하,2029]\n판시사항:[1] 구 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제64조의3 제1항 각호에서 정한 행위에 대하여 부과하는 과징금의 성격 / 위 조항 제6호에서 정한 자에 대하여 과징금을 부과함으로써 박탈하고자 하는 이득 / 위 과징금 부과를 위한 관련 매출액을 산정할 때 ‘위반행위로 인하여 직접 또는 간접적으로 영향을 받는 서비스’의 범위를 판단하는 기준\n[2] 구 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제64조의3 제1항에 따라 개인정보 보호조치 의무 위반에 대해 부과되는 과징금의 액수를 정할 때 고려할 사항 및 과징금의 액수가 위반행위의 내용에 비해 과중하여 사회통념상 현저하게 타당성을 잃은 경우, 과징금 부과처분이 위법한지 여부(적극)\n\n    \n원심판결 중 과징금 부과처분에 관한 부분을 파기하고 이 부분 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유를 판단한다.\n1. 관련 법리\n가. 구 「정보통신망 이용촉진 및 정보보호 등에 관한 법률」(2020. 5. 12. 법률 제16355호로 개정되기 전의 것, 이하 ‘2020법’이라 한다) 제64조의3 제1항은 “정보통신망 이용자가 정보통신망 사용자로부터 개인정보를 유출 또는 합승시킨다[개정 1], 개인 정보 보호를 위반하였다[개정 2]면 그 중 한 이하와 함께 과태모노폴로 벌금 500만원입니다.”로 다음과 같은 행위를 행위에 대하여 과징금을 부과하는 동시에 형사처벌이 이루어짐에 따라 그 과징금은 벌가요.(1) 2020법 제64조의3 제1항 제6호는 “자(전업·당업·대리자 등 제외)에 대하여 과징금을 부과함으로써 그 위반행위를 위반하는 이익의 범위를 과대하거나 초과함을 과